callback : epoch끝나고 수행하는 함수 : reaction 이라고 생각하면 됨  
        
decorator, 상속, duck typing으로 추가기능 구현가능  

callback때마다 메일이나 핸드폰으로 결과 보낼 수 있다.  

scikit에서는 hold out하기 위해서 train test split 해야함
keras는 validation_split으로 hold out가능

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()
# 판다스 쓰는 이유 == x축이 index이다. && 그래프 그리기 쉬워서
# https://www.tensorflow.org/tutorials/keras/basic_regression

오버피팅 막기
1. 앙상블 모델 
2. early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)  
patience값 정하기 어렵다. == activation바꿔서  == callback에서 사용  
: val_loss가 10번 안바뀌면 학습 중단 == patience == 참는다. :  
3. regularization : 모델이 크게 변하는걸 막아줌 : layer안에서 수행
4. drop-out : 몇개를 없앤다던데 뭐지 : layer를 추가해서 : 몇 개 노드를 생략해서 학습하겠다.
5. 네트워크 용량을 줄인다.(노드, layer줄임)

성능평가할 때,  
제약 조건도 표시해야 한다.

In [2]:
import tensorflow as tf

In [9]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data()

In [13]:
inversed = {j:i for i,j in tf.keras.datasets.imdb.get_word_index().items()}
# 숫자값을 넣어서 원하는 단어를 추측할 수 있다.

In [12]:
inversed

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [27]:
y_train[0]
# 0번째 단어는 긍정을 의미함

1

In [26]:
[inversed[i] for i in X_train[0]]
# 이런 단어가 있으면 긍정이구나...!

['the',
 'as',
 'you',
 'with',
 'out',
 'themselves',
 'powerful',
 'lets',
 'loves',
 'their',
 'becomes',
 'reaching',
 'had',
 'journalist',
 'of',
 'lot',
 'from',
 'anyone',
 'to',
 'have',
 'after',
 'out',
 'atmosphere',
 'never',
 'more',
 'room',
 'titillate',
 'it',
 'so',
 'heart',
 'shows',
 'to',
 'years',
 'of',
 'every',
 'never',
 'going',
 'villaronga',
 'help',
 'moments',
 'or',
 'of',
 'every',
 'chest',
 'visual',
 'movie',
 'except',
 'her',
 'was',
 'several',
 'of',
 'enough',
 'more',
 'with',
 'is',
 'now',
 'current',
 'film',
 'as',
 'you',
 'of',
 'mine',
 'potentially',
 'unfortunately',
 'of',
 'you',
 'than',
 'him',
 'that',
 'with',
 'out',
 'themselves',
 'her',
 'get',
 'for',
 'was',
 'camp',
 'of',
 'you',
 'movie',
 'sometimes',
 'movie',
 'that',
 'with',
 'scary',
 'but',
 'pratfalls',
 'to',
 'story',
 'wonderful',
 'that',
 'in',
 'seeing',
 'in',
 'character',
 'to',
 'of',
 '70s',
 'musicians',
 'with',
 'heart',
 'had',
 'shadows',
 'they'

In [ ]:
핫은 1을 의미
원 - 핫 : 일이 하나
멀티 - 핫 : 일이 여러개

In [30]:
import numpy as np

In [31]:
NUM_WORDS = 10000

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=NUM_WORDS)

# 자연어 처리에서 쓰는 기법
# 값의 크기의 영향을 최소화 시킬 수 있다. == one-hot , multi-hot쓰는 이유
def multi_hot_sequences(sequences, dimension):
    # 0으로 채워진 (len(sequences), dimension) 크기의 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0  # results[i]의 특정 인덱스만 1로 설정합니다
        # numpy에서만 가능한 indexing방식  ","가능한가 확인해보자
        
    return results


train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)

1. 데이터 많을수록 좋다. == 회사에 데이터가 충분히 많은지 어필해야함

안타깝지만 어떤 모델의 (층의 개수나 뉴런 개수에 해당하는) 적절한 크기나 구조를 결정하는 마법같은 공식은 없습니다.  
여러 가지 다른 구조를 사용해 실험을 해봐야만 합니다. == 정해진 공식이 없다. == No free lunch  

실험을 해봐서 찾아봐야 한다. 정답이 없으므로 전략이 필요하다.   
[구글의 학습 전략](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit)  

데이터별로 no free lunch

In [36]:
baseline_model = tf.keras.Sequential([
    # `.summary` 메서드 때문에 `input_shape`가 필요합니다 : 가 아니고
    # input_shape하면 input layer만들어줌
    tf.keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(NUM_WORDS,)),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    # relu를 쓰면 sigmoid보다 층이 많아도 학습이 잘된다. == gradient vanishing안 일어남
    
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])
# 정확도로만 성능 측정하면 overfitting문제가 있다.

# loss는 바꾸는게 아니다.
# crossentroy 정보 이동량

baseline_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
목적에 따라서 정확도를 따져봐야 함
# 75% ~ 85% 정도 기업에서 사용

In [ ]:
from sklearn.linear_model import Log

In [ ]:
모델의 복잡성을 줄이면 == 간단해짐 == bias낮아짐 == variance가 높아짐 == 일반성을 가짐 == 최적의(일반성) 모델

duplicated data는 중요성은 강조되지만, parameter찾을때는 도움이 안된다.  

In [ ]:
# visualization 목적 : 
그래프로 검증할 수 있음 == 숫자로만 보면 한번에 알아보기 힘들다.

데이터 형태에서 어떻게 분리할 수 있는지 테스트해보는 사이트  
[tensorflow playground](https://playground.tensorflow.org/#activation=relu&batchSize=10&dataset=spiral&regDataset=reg-plane&learningRate=0.1&regularizationRate=0&noise=0&networkShape=4,4,4,2&seed=0.60058&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=true&xSquared=true&ySquared=true&cosX=false&sinX=true&cosY=false&sinY=true&collectStats=false&problem=classification&initZero=false&hideText=false)

In [ ]:
설치와 임포트
필요한 라이브러리를 설치하고 텐서플로를 임포트(import)합니다:

!pip install -q h5py pyyaml
모델을 위와 같은 방식으로 저장 가능해짐

In [ ]:
Non-trainable params : 사람의 지식으로 param을 상수로 만듦 == 더 빠를 수 있다.

In [42]:
import os

import tensorflow as tf
from tensorflow import keras

In [43]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [45]:
# 간단한 Sequential 모델을 반환합니다
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
    return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [46]:
checkpoint_path = "cp.ckpt" #피클 확장자 안 중요하다. 
# 이름이 같아서 체크포인트가 하나만 생성
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])

W0625 22:08:37.519097 4785006016 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 672/1000 [===================>..........] - ETA: 0s - loss: 1.3971 - accuracy: 0.5908
Epoch 00001: saving model to cp.ckpt
1000/1000 [==============================] - 1s 862us/sample - loss: 1.1713 - accuracy: 0.6620 - val_loss: 0.7567 - val_accuracy: 0.7630
Epoch 2/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.4638 - accuracy: 0.8767
Epoch 00002: saving model to cp.ckpt
1000/1000 [==============================] - 0s 454us/sample - loss: 0.4303 - accuracy: 0.8850 - val_loss: 0.5249 - val_accuracy: 0.8280
Epoch 3/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.2875 - accuracy: 0.9253
Epoch 00003: saving model to cp.ckpt
1000/1000 [==============================] - 0s 389us/sample - loss: 0.2837 - accuracy: 0.9230 - val_loss: 0.4816 - val_accuracy: 0.8450
Epoch 4/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.2283 - accuracy: 0.9427
Epoch 00004: saving model to cp.ckpt
1

cp.ckpt.data-00000-of-00001, 
cp.ckpt.index는 binary파일이라서 읽을 수 없다. 

In [ ]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

# 특점 시점의 모델을 가져올 수 있게된다. == overfitting이 일어나기 전의 모델을 가져올 수 있다.

In [ ]:
전체 모델을 저장하는 기능은 매우 유용합니다.   
TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다(HDF5, Saved Model).   
또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수 있습니다(HDF5, Saved Model).  

new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

느린 학습은 컴퓨터에서 하고, predict는 빠르니까 폰,웹에서 모델을 불러올 수 있다.  
(안드, 웹에서 학습하긴 힘드니까..)